In [1]:
import pandas as pd
import numpy as np
import re
from nltk.corpus import stopwords
import json
from nltk.tokenize import  word_tokenize
import tensorflow as tf
import ast

/home/pratik/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:

max_sequence_length = 20

In [3]:
raw_data = pd.read_csv('/home/pratik/NER/data/large_positive.csv')
len(raw_data)

21682

In [4]:
raw_data.drop_duplicates(subset=['body'], inplace=True)
len(raw_data)

16623

In [5]:
def convert_to_list(original_text_list):
    l = ast.literal_eval(original_text_list)
    l = [i.strip() for i in l]
    return l
raw_data['original_text'] = raw_data['original_text'].apply(lambda x: convert_to_list(x))

In [6]:
def generate_y(text, original_text_list, k):
    y = np.zeros([max_sequence_length, 2])
    try:
        for i in range(len(y)):
            y[i][0] = 1
        for original_text_words in original_text_list:
            for original_text in word_tokenize(original_text_words):
                words = word_tokenize(text)
                for i in range(len(words)):
                    if words[i].lower() == original_text.lower():
                      #  print(words[i], original_text)
                        y[i][1] = 1
                        y[i][0] = 0
    except:
        pass
    return y


In [7]:
y_list = np.zeros([len(raw_data), max_sequence_length, 2])
for body, original_text_list, i in zip(raw_data['body'], raw_data['original_text'], range(len(raw_data))):
    
    y_list[i] = generate_y(body, original_text_list, i)

In [8]:
len(raw_data), len(y_list)

(16623, 16623)

In [9]:
word_tokenize(raw_data['body'].iloc[11]), raw_data['original_text'].iloc[11], y_list[11]

(['No', 'I', 'ask', 'Paytm', 'debit', 'card', 'queries'],
 ['paytm debit'],
 array([[1., 0.],
        [1., 0.],
        [1., 0.],
        [0., 1.],
        [0., 1.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.]]))

In [10]:
char_vocab = []
def get_character_set(text):
    for char in text:
        if not char in char_vocab:
            char_vocab.append(char)

raw_data['body'].apply(lambda x: get_character_set(x))

0        None
1        None
2        None
3        None
5        None
6        None
7        None
8        None
9        None
10       None
11       None
12       None
13       None
14       None
15       None
16       None
17       None
18       None
19       None
20       None
21       None
22       None
23       None
24       None
25       None
26       None
27       None
28       None
29       None
30       None
         ... 
21287    None
21288    None
21289    None
21290    None
21291    None
21292    None
21293    None
21294    None
21295    None
21296    None
21297    None
21298    None
21299    None
21301    None
21302    None
21304    None
21305    None
21306    None
21307    None
21309    None
21310    None
21311    None
21312    None
21313    None
21316    None
21318    None
21319    None
21320    None
21321    None
21322    None
Name: body, Length: 16623, dtype: object

In [11]:
char_vocab_len = len(char_vocab)
char_one_hot = np.zeros([char_vocab_len, char_vocab_len])
for i in range(len(char_vocab)):
    char_one_hot[i][i] = 1
    

In [12]:
max_char_length = 16
max_sequence_length = 20
char_dimensions = char_vocab_len
batch_size = 100
raw_data_len = len(raw_data)
ids_char = np.zeros([(raw_data_len),max_sequence_length,max_char_length ])
ids_char = ids_char + len(char_vocab) -1

In [13]:
g = 0
for i in (range((raw_data_len))):
    body = raw_data.iloc[i]['body']
    words = word_tokenize(body)
    for j in (range(len(words))):
        word = words[j]
        for k in range(len(word)):
            char = word[k]
            try:
                ids_char[i][j][k] = char_vocab.index(char)
            except:
                pass

In [14]:
ids_char.shape

(16623, 20, 16)

In [15]:
import tensorflow as tf
tf.reset_default_graph()


In [16]:
#weights = {'W_conv1': tf.get_variable("W_conv1", shape=([5,5,1,32]), dtype=tf.float32,
#              initializer=tf.random_normal_initializer()),
#           'W_conv2':                tf.get_variable("W_conv2", shape=([5,5,32,64]), dtype=tf.float32,
#              initializer=tf.random_normal_initializer()),
#           'W_fc':tf.get_variable("W_fc", shape=([(int(max_char_length//4))*(int(char_dimensions//4))*64,1024]), dtype=tf.float32,
#              initializer=tf.random_normal_initializer())
#           }



#biases = {'b_conv1':tf.get_variable("b_conv1", shape=([32]), dtype=tf.float32,
#              initializer=tf.random_normal_initializer()),
#           'b_conv2':tf.get_variable("b_conv2", shape=([64]), dtype=tf.float32,
#              initializer=tf.random_normal_initializer()),
#           'b_fc':tf.get_variable("b_fc", shape=([1024]), dtype=tf.float32,
#              initializer=tf.random_normal_initializer())
#           }
weights = {'W_conv1':tf.Variable(tf.random_normal([5,5,1,32])),
           'W_conv2':tf.Variable(tf.random_normal([5,5,32,64])),
           'W_fc':tf.Variable(tf.random_normal([(int(10//2))*(int(4//2))*32,100])),
           'out':tf.Variable(tf.random_normal([100, 10]))}

biases = {'b_conv1':tf.Variable(tf.random_normal([32])),
           'b_conv2':tf.Variable(tf.random_normal([64])),
           'b_fc':tf.Variable(tf.random_normal([100])),
           'out':tf.Variable(tf.random_normal([10]))}
n_nodes_hl1 = 40
hidden_1_layer = {'weights':tf.Variable(tf.random_normal([max_char_length * char_dimensions, n_nodes_hl1])),
                          'biases':tf.Variable(tf.random_normal([n_nodes_hl1]))}


def character_level_embedding(char_input_data):
    print('sdiugfh', char_input_data)
    char_input_data = tf.cast(char_input_data, dtype=tf.int32)
    char_one_hot_tensor = tf.convert_to_tensor(char_one_hot, dtype=tf.int32)
    char_data = tf.placeholder(shape = (max_sequence_length,max_char_length), dtype=tf.int32)
    char_data = tf.nn.embedding_lookup(char_one_hot_tensor,char_input_data)
    print('char_data', char_data)
    keep_rate = 1
    print('aya')
    
    #print(char_data.shape)
#The place holder to hold the values of new x and y at different times.
    x = tf.placeholder(dtype=tf.float32, shape=[max_sequence_length, max_char_length * char_dimensions])
    x = tf.cast(tf.reshape(char_data, shape=[max_sequence_length, max_char_length * char_dimensions]), dtype=tf.float32)
    #print(x)
#y = tf.placeholder('float', [batch_size, n_classes])

    def embedding_layer(data):
        print('aya')
        l1 = tf.add(tf.tensordot(data,hidden_1_layer['weights'], axes=1), hidden_1_layer['biases'])
        l1 = tf.nn.relu(l1)
        print(l1.shape)
        return l1

    def conv2d(x, W):
        return tf.nn.conv2d(x, W, strides=[1,1,1,1], padding='SAME')

    def maxpool2d(x):
        #                        size of window         movement of window
        return tf.nn.max_pool(x, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')


    def convolutional_neural_network(x):
        
#        tf.get_variable("W_conv1", shape=([5,5,1,32]), dtype=tf.int32,
#                      initializer=tf.ones_initializer())
#                tf.get_variable("W_conv2", shape=([5,5,32,64]), dtype=tf.int32,
#                      initializer=tf.ones_initializer())
#        
#                        tf.get_variable("W_fc", shape=([(int(max_char_length//4))*(int(char_dimensions//4))*64,1024]), dtype=tf.int32,
#                      initializer=tf.ones_initializer())

        
        

        print('og', x.shape)
        x = tf.reshape(x, shape=[max_sequence_length,10,4, 1])
        
        conv1 = tf.nn.relu(conv2d(x, weights['W_conv1']) + biases['b_conv1'])
        conv2 = maxpool2d(conv1)

        #conv2 = tf.nn.relu(conv2d(conv1, weights['W_conv2']) + biases['b_conv2'])
        #conv2 = maxpool2d(conv2)
        
        print('con2',conv2.shape)
        fc = tf.reshape(conv2,[max_sequence_length, (10//2) * (4//2) * 32])
        fc = tf.nn.relu(tf.matmul(fc, weights['W_fc'])+biases['b_fc'])
        fc = tf.nn.dropout(fc, keep_rate)
        #print(fc.shape)
        return fc

    def train_neural_network(x):    
        embedded_x = embedding_layer(x)
        print(embedded_x)
        #embedded_x = tf.reshape(x, [max_sequence_length, char_dimensions])
        convoluted_x = convolutional_neural_network(embedded_x)
        
        return convoluted_x
    
    char_data = train_neural_network(x)
    return char_data


char_input_data = tf.placeholder(dtype=tf.float32,shape= [batch_size, max_sequence_length, max_char_length])
char_embeddings = tf.map_fn(lambda x: character_level_embedding(x), char_input_data, dtype=tf.float32)



sdiugfh Tensor("map/while/TensorArrayReadV3:0", shape=(20, 16), dtype=float32)
char_data Tensor("map/while/embedding_lookup:0", shape=(20, 16, 158), dtype=int32)
aya
aya
(20, 40)
Tensor("map/while/Relu:0", shape=(20, 40), dtype=float32)
og (20, 40)
con2 (20, 5, 2, 32)


In [17]:
init_g = tf.global_variables_initializer()
init_l = tf.local_variables_initializer()
with tf.Session() as sess:
    sess.run(init_g)
    sess.run(init_l)
    p = ids_char.astype(int)[100:200]
    print(type(p[0][0][0]))
    b = sess.run([char_embeddings], feed_dict={char_input_data: p})
    print(b[0].shape)
    

<class 'numpy.int64'>
(100, 20, 100)


In [19]:
lstmUnits = 64
numClasses = 2
epochs = 10
numDimensions = 100
batchSize = 100
train_len = 14000

y_train = y_list[:train_len]
y_test = y_list[train_len:]
ids_char_train = ids_char[:train_len]
ids_char_test = ids_char[train_len:]




#from sklearn.model_selection import train_test_split
#X_train, X_test, y_train, y_test, ids_char_train, ids_char_test = train_test_split(ids,y_list,ids_char, test_size=.2)

len(y_test), len(ids_char_train)

(2623, 14000)

In [20]:
def getTrainBatch(j):
    i = j*batchSize
    labels = y_train[i:i+batchSize]
    arr_char = ids_char_train[i:i+batchSize]
    #print(labels[0])
    return  labels, arr_char

In [21]:
def getTestBatch(j):
    i = j*batchSize
    labels =   y_test[i:i+batchSize]
    arr_char = ids_char_test[i:i+batchSize]
    return  labels, arr_char

In [22]:
labels = tf.placeholder(tf.float32, [batchSize, max_sequence_length, numClasses])


In [24]:
print(char_embeddings.shape)
data2 = tf.placeholder(shape = (batchSize,max_sequence_length,numDimensions+char_dimensions), dtype=tf.float32)
data2 = char_embeddings

(100, 20, 100)


In [25]:
fw_cell3 = tf.nn.rnn_cell.MultiRNNCell([tf.nn.rnn_cell.BasicLSTMCell(lstmUnits, state_is_tuple=True) for _ in range(2)])
bw_cell3 = tf.nn.rnn_cell.MultiRNNCell([ tf.nn.rnn_cell.BasicLSTMCell(lstmUnits, state_is_tuple=True) for _ in range(2)])

In [26]:
outputs,value2 = tf.nn.bidirectional_dynamic_rnn(fw_cell3, bw_cell3,data2,dtype=tf.float32)

In [27]:
#print(outputs)

outputs = tf.concat(outputs, 2)
#print(outputs.shape)

In [28]:
weight = tf.Variable(tf.truncated_normal([lstmUnits*2,numClasses]))
bias = tf.Variable(tf.constant(0.1, shape=[batchSize,max_sequence_length,numClasses]))

In [29]:
prediction = (tf.tensordot(outputs, weight, axes=((2,),(0,))) + bias)
print(prediction.shape)
p2 = tf.argmax(labels, axis=2)
print('p2', p2.shape)
print(labels)

prediction = tf.cast(prediction, dtype=tf.float32)

p2 = tf.cast(p2, dtype=tf.int32)

sequence_lengths = tf.constant(shape=[batch_size], value=np.zeros(batch_size)+max_sequence_length, dtype=tf.int32)


log_likelihood, transition_params = tf.contrib.crf.crf_log_likelihood(
inputs=prediction, 
sequence_lengths=sequence_lengths,
tag_indices=p2)
crf_loss = tf.reduce_mean(-log_likelihood)

prediction_arg = tf.argmax(prediction,axis=2)
label_arg = tf.argmax(labels,axis=2)

#false_positives
false_positives = tf.metrics.false_positives(predictions=prediction_arg, labels=label_arg)
false_negatives = tf.metrics.false_negatives(predictions=prediction_arg, labels=label_arg)
true_positives = tf.metrics.true_positives(predictions=prediction_arg, labels=label_arg)
true_negatives = tf.metrics.true_negatives(predictions=prediction_arg, labels=label_arg)
prec = tf.metrics.precision(predictions=prediction_arg, labels=label_arg)
rec = tf.metrics.recall(predictions=prediction_arg, labels=label_arg)
#false_positives = 1
init_g = tf.global_variables_initializer()
init_l = tf.local_variables_initializer()
#with tf.Session() as sess:
    #tf.initialize_all_variables().run()
#    sess.run(init_g)
#    sess.run(init_l)
#    nextBatch, nextBatchLabels, nextChars = getTrainBatch(2)
        #print(j/2)
#    pred, pa, la, fp, fn, tp, tn = sess.run([prediction, prediction_arg, label_arg, false_positives, false_negatives, true_positives, true_negatives], {input_data: nextBatch, labels: nextBatchLabels, char_input_data: nextChars})
#    print(pa.shape, la.shape, fp, fn, tp, tn)
    #a = pred[0]
    #b = nextBatchLabels
   # print(a.shape,b.shape)
    #print(a.shape)
    #print(np.argmax(a, axis=2)[0])
    #print(np.argmax(b, axis=2)[0])
    #print(np.logical_and(np.argmax(a, axis=2)[0], np.argmax(b, axis=2)[0] ))
    #print((np.equal(np.argmax(a, axis=2), np.argmax(b, axis=2)))[0])
    #print(np.mean(np.equal(np.argmax(a, axis=2), np.argmax(b, axis=2)), axis=1))
    #print(np.mean(np.equal(np.argmax(a, axis=2), np.argmax(b, axis=2)), axis=0))
    #print(np.mean(np.equal(np.argmax(a, axis=2), np.argmax(b, axis=2))))

correctPred = tf.equal(tf.argmax(prediction,axis=2), tf.argmax(labels,axis=2))
accuracy = tf.reduce_mean(tf.cast(correctPred, tf.float32))


(100, 20, 2)
p2 (100, 20)
Tensor("Placeholder_1:0", shape=(100, 20, 2), dtype=float32)


In [30]:
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=prediction, labels=labels))
optimizer = tf.train.AdamOptimizer().minimize(crf_loss)


Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See tf.nn.softmax_cross_entropy_with_logits_v2.



In [31]:
#with tf.Session() as sess:
#    sess.run(tf.global_variables_initializer())#

#    o,w,b,p = sess.run([outputs,weight,bias,prediction],feed_dict={input_data: X_train[:100], labels: y_train[:100]})
#    print(np.array(o).shape,np.array(w).shape,np.array(b).shape,np.array(p).shape)

In [32]:
import datetime

tf.summary.scalar('Loss', loss)
tf.summary.scalar('Accuracy', accuracy)
merged = tf.summary.merge_all()
logdir = "tensorboard/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S") + "/"
writer = tf.summary.FileWriter(logdir, sess.graph)

In [36]:
sess = tf.InteractiveSession()
saver = tf.train.Saver()
init_g = tf.global_variables_initializer()
init_l = tf.local_variables_initializer()
sess.run(init_g)
sess.run(init_l)

for i in range(epochs):
    print("epoch:  "+str(i))
    #Next Batch of reviews
    losses = 0
    for j in range(len(y_train)//batchSize):
        nextBatchLabels, nextChars = getTrainBatch(j);
        #print(j/2)
        ll, _,pred,lab = sess.run([loss, optimizer,prediction,labels], { labels: nextBatchLabels, char_input_data: nextChars})
        print(ll)
        losses += ll

    #Write summary to Tensorboard
        if (j % 5 == 0):
            summary = sess.run(merged, { labels: nextBatchLabels, char_input_data: nextChars})
            writer.add_summary(summary, j)

        #Save the network every 10,000 training iterations
        if (j % 50 == 0 and j != 0):
            save_path = saver.save(sess, "models/pretrained_lstm.ckpt", global_step=i)
            print("saved to %s" % save_path)
    print(losses)
print((np.array(pred)))
x3 = np.array(lab)
writer.close()

epoch:  0
0.3987624
0.90014136
0.17772464
0.15797955
0.19017032
0.18491602
0.1460002
0.15008932
0.21395437
0.23473258
0.24925701
0.19423728
0.1650892
0.1571514
0.15309298
0.15994409
0.1546994
0.1463634
0.17042765
0.16593762
0.202456
0.17986514
0.17665532
0.14859912
0.12116726
0.14190376
0.12589231
0.13053177
0.14309208
0.1578175
0.18094932
0.17227429
0.17923756
0.15482555
0.15145169
0.16277939
0.13959494
0.15938716
0.15751025
0.15443088
0.13388714
0.13238572
0.15260279
0.13956513
0.15441099
0.13182399
0.11666346
0.12186949
0.1189976
0.11131427
0.17885995
saved to models/pretrained_lstm.ckpt-0
0.15972558
0.15358777
0.1477355
0.1450481
0.1525245
0.14736769
0.11010355
0.14495264
0.13990963
0.15209074
0.1370285
0.13218169
0.17902768
0.110628545
0.11866246
0.12079778
0.12195114
0.12037362
0.16814762
0.121802
0.12418744
0.11003621
0.11903722
0.08541898
0.09016533
0.12799452
0.10733378
0.09394478
0.08504093
0.089571275
0.094358645
0.09809705
0.09885243
0.10279909
0.09600934
0.09111594
0.12349

0.06630539
0.078332044
0.10955892
0.109565265
0.09381987
0.08560683
0.060701568
0.07729092
0.07716895
0.081196584
0.06695546
0.065221906
10.216587029397488
epoch:  5
0.07609995
0.07812386
0.05031038
0.058201946
0.06521308
0.09605107
0.06988258
0.06955486
0.07170533
0.04988722
0.047380757
0.06712038
0.07741045
0.12049396
0.08988285
0.0929103
0.077171475
0.06895026
0.06695694
0.054021068
0.073115
0.07428193
0.064942084
0.057807542
0.054739464
0.056462444
0.06465581
0.05603669
0.059050843
0.060461912
0.07209761
0.060567316
0.096738085
0.078242354
0.076584734
0.06816743
0.05934643
0.074183926
0.066192105
0.05652717
0.056683008
0.075352296
0.08346431
0.06013536
0.06873602
0.067696124
0.05021601
0.057692032
0.06672275
0.05166999
0.08781477
saved to models/pretrained_lstm.ckpt-5
0.084002964
0.07457636
0.065584876
0.068241656
0.07416856
0.08732222
0.06263323
0.096191615
0.075262785
0.08615224
0.06979013
0.08248529
0.09704475
0.06517871
0.070643604
0.08918947
0.077460304
0.08370929
0.112131685


0.058783505
0.08644637
0.07464417
0.075867936
0.05414255
0.04774733
0.04855007
0.04696612
0.054242805
0.046159245
0.058925536
0.061207686
0.06025599
0.07233694
0.06208997
0.06116723
0.07264536
0.07977136
0.063959055
0.079195544
0.049042333
0.080742374
0.047600675
0.050849084
0.05826159
0.06709718
0.095023066
0.09046025
0.080984354
0.075066745
0.061291244
0.066696785
0.059710402
0.06610486
0.05597218
0.06044088
8.874776564538479
[[[-2.7600276  -0.12708469]
  [ 0.64840394 -5.225586  ]
  [ 0.8027767  -6.0649643 ]
  ...
  [ 3.1541827  -9.728262  ]
  [ 3.284893   -9.447916  ]
  [ 3.4471755  -8.900519  ]]

 [[ 0.03037421 -2.4251707 ]
  [-1.5150591  -1.5322667 ]
  [-1.2128679  -1.8802137 ]
  ...
  [ 3.056615   -8.935806  ]
  [ 3.1736448  -8.859985  ]
  [ 3.4002907  -8.525463  ]]

 [[-0.9916136  -2.097333  ]
  [-2.3017015  -1.8869021 ]
  [-3.2879345  -0.09956159]
  ...
  [ 3.2628078  -9.542273  ]
  [ 3.3704545  -9.29285   ]
  [ 3.5119162  -8.770099  ]]

 ...

 [[-0.91885084 -1.9983137 ]
  [-2.

In [37]:
test_batches = 25
fin_pred = np.zeros([test_batches*100, 20, 1])
jj = 0
for j in range(test_batches):
    jj = j * 100
    nextBatchLabels, nextChars = getTestBatch(j);
    p, t = sess.run([prediction,transition_params], {labels: nextBatchLabels, char_input_data: nextChars})
    
    for k in range((batchSize)):
        viterbi_sequence, viterbi_score = tf.contrib.crf.viterbi_decode(p[k], t)
        print(np.array(viterbi_sequence).shape)
        viterbi_sequence = np.array(viterbi_sequence).reshape(20,1)
        fin_pred[jj:jj+100][k] = viterbi_sequence

(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,

(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,

In [38]:
prediction_analysis = {'original_text': [], 'pred_original_text': [], 'body': []}
#for i in range(len(y_test)):
for i in range((test_batches*100)):
    body = word_tokenize(str(raw_data[train_len:].iloc[i]['body']))
    lab_index_test = np.where(np.argmax(y_test, axis=2)[i]==1)[0]
    lab_index_pred = np.where(fin_pred[i]==1)[0]
    original_text = []
    pred_original_text = []
    for j in lab_index_test:
        original_text.append(body[j])
    for j in lab_index_pred:
        pred_original_text.append(body[j])

    prediction_analysis['original_text'].append(original_text)
    prediction_analysis['pred_original_text'].append(pred_original_text)
    prediction_analysis['body'].append(body)

In [39]:
prediction_analysis_pandas = pd.DataFrame.from_dict(prediction_analysis)

In [40]:
tru_pos = 0
fals_pos = 0 
fals_neg = 0
true_pos = []
false_pos = []
false_neg = []
def check_tp(original_text, pred_original_text):
    global tru_pos, fals_pos, fals_neg
    #print((set(original_text)))
    #print((set(pred_original_text)))
    og = set(original_text)
    pr = set(pred_original_text)
    res = og.intersection(pr)
    true_pos.append(res)
    
    false_pos.append(pr - res)
    false_neg.append(og - res)
    tru_pos += len((set(original_text).intersection(pred_original_text)))
    fals_pos += len(pr -res)
    fals_neg += len(og - res)
    
for i in range(2500):
    check_tp(prediction_analysis_pandas.iloc[i]['original_text'], prediction_analysis_pandas.iloc[i]['pred_original_text'])

In [41]:
pp = tru_pos/ (tru_pos + fals_pos)
rr = tru_pos/ (tru_pos + fals_neg)
tru_pos, fals_pos, fals_neg, pp, rr, (2*pp*rr)/(pp+rr)

(3850, 765, 586, 0.8342361863488624, 0.8678990081154193, 0.8507347254447022)

In [42]:
prediction_analysis_pandas['false_positives'] = false_pos
prediction_analysis_pandas['true_positives'] = true_pos
prediction_analysis_pandas['false_negatives'] = false_neg

In [44]:
prediction_analysis_pandas

,body,original_text,pred_original_text,false_positives,true_positives,false_negatives
0,"[i, call, on, just, dial, .., its, better, the...",[samsung],"[its, better]","{better, its}",{},{samsung}
1,"[i, want, ambala, bus, stand, context, no, .]","[bus, stand]",[],{},{},"{bus, stand}"
2,"[Beauty, parlour, for, women]","[Beauty, parlour]","[Beauty, parlour]",{},"{Beauty, parlour}",{}
3,"[Its, not, a, night, club, dear]","[night, club]","[night, club]",{},"{club, night}",{}
4,"[More, Nearby, :, workshop]",[workshop],[workshop],{},{workshop},{}
5,"[Not, send, me, agai]",[agai],[],{},{},{agai}
6,"[Car, workshop]","[Car, workshop]","[Car, workshop]",{},"{Car, workshop}",{}
7,[Daraganj],[Daraganj],[Daraganj],{},{Daraganj},{}
8,"[Look, for, him]","[for, him]","[Look, him]",{Look},{him},{for}
9,"[Nearby, sbi, branch]","[sbi, branch]","[sbi, branch]",{},"{sbi, branch}",{}
